**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. 
Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. 
Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [2]:
import pandas as pd

In [3]:
ratings = pd.read_csv('ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [4]:
ratings_gr = ratings.groupby('userId').agg({'timestamp': ['min', 'max'], 'rating': 'count'})
ratings_gr['life_time'] = ratings_gr['timestamp']['max'] - ratings_gr['timestamp']['min']
ratings_gr = ratings_gr[ratings_gr.rating.values > 100]
ratings_gr.head()

timestamp             rating life_time
               min         max  count          
userId                                         
1        964980499   965719662    232    739163
4        945078428  1007574542    216  62496114
6        845553109   845556915    314      3806
7       1106635416  1176181731    152  69546315
10      1455301553  1455619275    140    317722

**Задание  2**

2. Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [5]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [6]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [7]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [8]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [14]:
# таблица с тремя типами выручки для каждого client_id без указания адреса клиента
revenue_without_adress = rzd.merge(auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer')
revenue_without_adress

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [17]:
# аналогичная таблица по типам выручки с указанием адреса клиента
revenue_with_adress = client_base\
    .merge(rzd, on = 'client_id', how = 'left')\
    .merge(auto, on = 'client_id', how = 'left')\
    .merge(air, on = 'client_id', how = 'left')
revenue_with_adress

,client_id,address,rzd_revenue,auto_revenue,air_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,57483.0,NaN
3,114,Мира 14,5774.0,83.0,NaN
4,115,ЗЖБИиДК 1,981.0,912.0,81.0
5,116,Строителей 18,NaN,4834.0,4.0
6,117,Панфиловская 33,NaN,98.0,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


**Задание 3**

3. В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. 
Т. е. для каждого user_id известна последовательность координат (широта/долгота), 
когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? 
Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. 
А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. 
Как это отразится на вопросе расчетах пункта 1?
- Какие дополнительные признаки можно получить из координат? 
Ведь это просто числа, которые сами по себе мало что дают.


Ответы:

3.1. Ответ: 
Как я поняла из условия задачи – координаты получаются только в тот момент, когда они требуются приложению для полноценной
работы, следовательно – связать их с таблицей покупок и визитов можно только при условии совпадения временных интервалов 
(там скорее всего придется делать именно интервалы, так как вероятность только что время совпадет 1 в 1 – маловероятно – 
 например, координаты могут понадобиться приложению не в момент входа пользователя в приложение 
 (что характерно для времени визита).
Для покупок этот интервал считается достаточно легко – как разница времени между входом в систему 
 (время визита) и временем покупки. А для посещений как посчитать правильно – не знаю…, возможно, 
 можно использовать только сопоставимость дат (получения геокоординат и посещений) (приняв гипотезу, что в течение одного 
дня пользователь вряд ли переедет в другой город)

 Если коротко – для связывания таблиц нужно дата и время для всех действий пользователя и дата и время снятия координат.

3.2. Ответ: 
Да, тут очень много проблем: во-первых, при снятии геокоординат в закрытых местах (например, в закрытых городах или возле Кремля) 
 мы будем получать совершенно «кривые данные», более того – они всегда будут разными. 
 Частично их отсеять можно, зная город пользователя (например который он указал при регистрации или который был
вычислен для него ранее – но думаю, что полностью такие случаи мы отсеять не сможем.
Во-вторых, один и тот же пользователь может просто заходить на сайт или совершать покупки в разных местах, например дома и 
на работе или по дороге на работу. Такие варианты можно вычислить например с использованием средне-статистических данных 
по времени пути пользователя на работу/с нее (получив расстояние и сравнив со статистическими данными). 
Более того, такие варианты будут иметь смысл только если пользователь живет например в Зеленограде и работает в Москве. 
В случае бОльших расстояний возможна следующая ситуация:
Точность геокоординат определяется тем. как долго пользователь находится в приложении. Возможно, разработчики смогут 
настроить получение геокоординат более точным способом, но это может сказаться на отклике системы для пользователя. 
Однако, в данном случае нужно иметь ввиду, что координаты, полученные при совершении покупки будут более точными, чем при визите, так как пользователь банально дольше использовал приложение во время покупки. 
Поэтому данные по городам точно не будут 100-% точными, но насколько – сказать очень трудно. 

3.3. Ответ: 
Зная координаты пользователя, можно сказать, в каком городе он находится – соответственно, получить данные о  статистике 
визитов и покупок, а также их конверсии в разрезе городов. 
Также можно получить информацию о том, где произошла покупка и выявлять триггеры (под триггерами в данному случае подразумеваю здания, организации…) – в дальнейшем можно увеличить конверсию покупок, размещая целевую рекламу именно в этих точках.